In [13]:
#Import modul yang digunakan
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import graphviz
from google.colab import files
from sklearn import preprocessing
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.tree import export_graphviz
from matplotlib import pyplot
from IPython.display import display
import statsmodels.api as sm
from scipy.stats import norm
import statsmodels.api as sm
from google.colab import files
from sklearn.utils import resample
import math


In [14]:
# Upload Dataset
uploaded = files.upload()

for nama_file in uploaded:
    data = pd.read_csv(nama_file)
    print(f"File {nama_file} berhasil diunggah dan diproses")

# Menentukan kolom-kolom yang digunakan untuk mengecek duplikat
kolom_duplikat = ['R101N', 'R102N', 'R103N', 'R104N']

# Mengubah isi kolom menjadi huruf kecil
data[kolom_duplikat] = data[kolom_duplikat].apply(lambda x: x.str.lower())

# Menghitung jumlah total data yang duplikat berdasarkan kolom tertentu
total_duplikat = data.duplicated(subset=kolom_duplikat).sum()
if total_duplikat > 0:
    print(f"Total data yang duplikat berdasarkan kolom {kolom_duplikat}: {total_duplikat}")
    print("Data yang duplikat:")
    print(data[data.duplicated(subset=kolom_duplikat)])
else:
    print(f"Tidak ada data yang duplikat berdasarkan kolom {kolom_duplikat}.")


Saving 300acak.csv to 300acak (1).csv
File 300acak (1).csv berhasil diunggah dan diproses
Tidak ada data yang duplikat berdasarkan kolom ['R101N', 'R102N', 'R103N', 'R104N'].


In [15]:
# Mengecek Missing Value
print("Mengecek Missing Value:")
print(data.isnull().sum())
print(data.isnull().sum().sum())

Mengecek Missing Value:
R101N       0
R102N       0
R103N       0
R104N       0
Target      0
R809A       0
R809B       0
R809C       0
R809D       0
R809E       0
R809F       0
R1205A1     0
R1205A2     0
R1205A3     0
R1206A1     0
R1206A2     0
R1206A3     0
R1206A4     0
R1007C      0
R1001C1     0
R1001A      0
R1007A      0
R1003B      0
R1004       0
R1005C      0
R1006B      0
R1401A     36
R1401C     36
R501C       0
R502A       0
R604A       0
R604B       0
R808A       0
R808B       0
dtype: int64
72


In [16]:
# Mendapatkan nilai modus dari kolom 'R1401A'
modus_kolom_R1401A = int(data['R1401A'].mode()[0])

# Mendapatkan nilai modus dari kolom 'R1401C'
modus_kolom_R1401C = int(data['R1401C'].mode()[0])

# Mengisi nilai yang kosong pada kolom 'R1401A' dengan nilai modus
data['R1401A'] = data['R1401A'].fillna(modus_kolom_R1401A)

# Mengisi nilai yang kosong pada kolom 'R1401C' dengan nilai modus
data['R1401C'] = data['R1401C'].fillna(modus_kolom_R1401C)

print(data.head())

              R101N            R102N          R103N      R104N  Target  R809A  \
0  sulawesi selatan        bulukumba   bonto bahari      a r a       1      1   
1              bali       karangasem          abang      abang       1      7   
2          bengkulu  bengkulu tengah  pondok kelapa  abu sakim       1      1   
3       jawa tengah         magelang       muntilan   adikarto       1      7   
4    sumatera utara        batu bara   medang deras  aek nauli       1      1   

   R809B  R809C  R809D  R809E  ...  R1005C  R1006B  R1401A  R1401C  R501C  \
0      1      1      1      0  ...       2       1     1.0     1.0      3   
1      1     14      5      3  ...       1       1     1.0     1.0      3   
2      1      1      7      0  ...       2       4     1.0     1.0      3   
3      7      0      6      1  ...       1       1     1.0     1.0      3   
4      1      0     10      0  ...       2       1     3.0     3.0      3   

   R502A  R604A  R604B  R808A  R808B  
0      2   

In [17]:
# Mengecek Missing Value
print("Mengecek Missing Value:")
print(data.isnull().sum())
print(data.isnull().sum().sum())

Mengecek Missing Value:
R101N      0
R102N      0
R103N      0
R104N      0
Target     0
R809A      0
R809B      0
R809C      0
R809D      0
R809E      0
R809F      0
R1205A1    0
R1205A2    0
R1205A3    0
R1206A1    0
R1206A2    0
R1206A3    0
R1206A4    0
R1007C     0
R1001C1    0
R1001A     0
R1007A     0
R1003B     0
R1004      0
R1005C     0
R1006B     0
R1401A     0
R1401C     0
R501C      0
R502A      0
R604A      0
R604B      0
R808A      0
R808B      0
dtype: int64
0


In [18]:
# Menentukan kolom yang akan dinormalisasi
kolom_normalize = data.columns[5:34]

# Melakukan Min-Max Normalization pada kolom tertentu
data[kolom_normalize] = (data[kolom_normalize] - data[kolom_normalize].min()) / (
    data[kolom_normalize].max() - data[kolom_normalize].min()
)

# Menampilkan hasil normalisasi
print(data.head())

              R101N            R102N          R103N      R104N  Target  \
0  sulawesi selatan        bulukumba   bonto bahari      a r a       1   
1              bali       karangasem          abang      abang       1   
2          bengkulu  bengkulu tengah  pondok kelapa  abu sakim       1   
3       jawa tengah         magelang       muntilan   adikarto       1   
4    sumatera utara        batu bara   medang deras  aek nauli       1   

      R809A     R809B     R809C     R809D     R809E  ...    R1005C  R1006B  \
0  0.013333  0.016667  0.027778  0.017857  0.000000  ...  0.333333     0.0   
1  0.093333  0.016667  0.388889  0.089286  0.214286  ...  0.000000     0.0   
2  0.013333  0.016667  0.027778  0.125000  0.000000  ...  0.333333     1.0   
3  0.093333  0.116667  0.000000  0.107143  0.071429  ...  0.000000     0.0   
4  0.013333  0.016667  0.000000  0.178571  0.000000  ...  0.333333     0.0   

   R1401A  R1401C  R501C  R502A  R604A  R604B  R808A  R808B  
0     0.0     0.0    1.0

In [19]:
# Memilih variabel dependen dari kolom 5 hingga 34
x = data.iloc[:, 5:34]

# Memilih variabel independen
y = data[['Target']]

In [20]:
# Gunakan StratifiedKFold sebagai objek cv
random_state = 42
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)

# Fungsi untuk menampilkan pembagian data train dan data test
def display_split_indices(cv, x, y):
    for train_idx, test_idx in cv.split(x, y):
        print("Data Train:", len(train_idx), "Data Test:", len(test_idx))

display_split_indices(cv, x, y)

Data Train: 1485 Data Test: 165
Data Train: 1485 Data Test: 165
Data Train: 1485 Data Test: 165
Data Train: 1485 Data Test: 165
Data Train: 1485 Data Test: 165
Data Train: 1485 Data Test: 165
Data Train: 1485 Data Test: 165
Data Train: 1485 Data Test: 165
Data Train: 1485 Data Test: 165
Data Train: 1485 Data Test: 165


In [21]:
# Pembagian Dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=random_state)

# Menampilkan bentuk dataset
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1485, 29), (165, 29), (1485, 1), (165, 1))

In [22]:
#Estimasi Parameter

def feature_selection_logistic(x, y, alpha=0.05):
    # Tetapkan seed untuk konsistensi
    np.random.seed(42)

    selected_features = []
    p_values = {}  # Dictionary untuk menyimpan p-value setiap fitur yang terpilih
    while len(selected_features) < len(x.columns):
        remaining_features = list(set(x.columns) - set(selected_features))
        best_pvalue = float('inf')
        best_feature = None
        for feature in remaining_features:
            model = sm.Logit(y, sm.add_constant(x[selected_features + [feature]])).fit(disp=0)
            pvalue = model.pvalues[feature]
            print("P-value of", feature, ":", pvalue)  # Menampilkan P-Value setiap fitur
            if pvalue < best_pvalue:
                best_pvalue = pvalue
                best_feature = feature
        if best_pvalue < alpha:
            selected_features.append(best_feature)
            p_values[best_feature] = best_pvalue  # Simpan p-value fitur terpilih
        else:
            break
    return selected_features, p_values


selected_features, p_values = feature_selection_logistic(x_train, y_train)
print("Fitur yang dipilih:", selected_features)

P-value of R809B : 0.03847555118331555
P-value of R1001C1 : 0.1948739189038472
P-value of R1005C : 6.520877939565111e-10
P-value of R502A : 0.6785148221904351
P-value of R1003B : 1.4264136639006517e-05
P-value of R1206A1 : 0.06213723472203469
P-value of R809F : 0.0005903907302694199
P-value of R1001A : 9.531698778755681e-07
P-value of R1205A1 : 0.0976721920249841
P-value of R808B : 0.16944737944357902
P-value of R809E : 0.0014292187097833552
P-value of R1205A2 : 0.8591392135302041
P-value of R1206A4 : 0.06410853475712973
P-value of R604A : 3.81574100376549e-06
P-value of R809C : 0.17627933651995298
P-value of R808A : 0.9582634303940821
P-value of R1004 : 1.0975304144980992e-05
P-value of R809A : 0.3333814394423602
P-value of R809D : 6.65097123822082e-05
P-value of R1206A3 : 0.000654018764618649
P-value of R1401A : 5.6402704156888426e-08
P-value of R1401C : 0.001269628082464303
P-value of R1006B : 1.0965455419058766e-14
P-value of R1206A2 : 0.31367958497073667
P-value of R1007C : 0.0031

In [23]:
# Mendapatkan selected_features
selected_features, _ = feature_selection_logistic(x_train, y_train)

# Menyiapkan data dalam bentuk dictionary
dataa = {"Feature": selected_features}

# Membuat DataFrame dari data
df = pd.DataFrame(dataa)

# Menyimpan DataFrame ke dalam file Excel
file_path = "fitur_dipilih.xlsx"
df.to_excel(file_path, index=False)

P-value of R809B : 0.03847555118331555
P-value of R1001C1 : 0.1948739189038472
P-value of R1005C : 6.520877939565111e-10
P-value of R502A : 0.6785148221904351
P-value of R1003B : 1.4264136639006517e-05
P-value of R1206A1 : 0.06213723472203469
P-value of R809F : 0.0005903907302694199
P-value of R1001A : 9.531698778755681e-07
P-value of R1205A1 : 0.0976721920249841
P-value of R808B : 0.16944737944357902
P-value of R809E : 0.0014292187097833552
P-value of R1205A2 : 0.8591392135302041
P-value of R1206A4 : 0.06410853475712973
P-value of R604A : 3.81574100376549e-06
P-value of R809C : 0.17627933651995298
P-value of R808A : 0.9582634303940821
P-value of R1004 : 1.0975304144980992e-05
P-value of R809A : 0.3333814394423602
P-value of R809D : 6.65097123822082e-05
P-value of R1206A3 : 0.000654018764618649
P-value of R1401A : 5.6402704156888426e-08
P-value of R1401C : 0.001269628082464303
P-value of R1006B : 1.0965455419058766e-14
P-value of R1206A2 : 0.31367958497073667
P-value of R1007C : 0.0031

In [24]:
# Inisialisasi model RandomForestClassifier dengan parameter tertentu
logreg = LogisticRegression(random_state=random_state)

# Inisialisasi GridSearchCV
penalty = ['l1', 'l2']
C = np.logspace(-2,2,10)

#Menjadikan ke dalam bentuk dictionary
hyperparameters = dict(penalty=penalty, C=C)

#Memasukan ke Grid Search
#CV itu Cross Validation
#Menggunakan 10-Fold CV
clf = GridSearchCV(logreg, hyperparameters, cv=10)
#Fitting Model
best_model = clf.fit(x_train[selected_features],y_train)
#Nilai hyperparameters terbaik
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

Best Penalty: l2
Best C: 4.6415888336127775


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
100 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be

In [25]:
# Mendapatkan nilai koefisien dari model terbaik
coefficients = best_model.best_estimator_.coef_

# Menampilkan nilai koefisien tanpa pembulatan
print("Nilai Koefisien:")
for i, feature in enumerate(selected_features):
    coef = coefficients[0][i]
    print(f"{feature}: {coef}")

Nilai Koefisien:
R1006B: -0.7224844195316218
R1005C: -1.351087838311801
R604A: -0.8929479705217073
R809D: 2.162025582318673
R1401A: -0.45747587442444443
R1001A: -0.5996345310620618


In [26]:
# Mendapatkan nilai intercept
intercept = best_model.best_estimator_.intercept_

# Menampilkan nilai intercept yang telah dibulatkan
print("Nilai intercept (𝛽0):", intercept)

Nilai intercept (𝛽0): [2.82535352]


In [32]:
y_train = y_train.values.ravel()

# Tetapkan seed untuk konsistensi
np.random.seed(42)  # Anda dapat mengganti 42 dengan angka apa pun

# Mendefinisikan jumlah bootstrap
n_bootstrap = 1000

# Inisialisasi list untuk menyimpan hasil bootstrap
coefficients_bootstrap = []

# Melakukan bootstrap
for _ in range(n_bootstrap):
    # Resampling data
    x_boot, y_boot = resample(x_train[selected_features], y_train)

    # Fitting model terbaik ke data bootstrapped
    model_boot = best_model.best_estimator_.fit(x_boot, y_boot)

    # Menyimpan koefisien
    coefficients_bootstrap.append(model_boot.coef_)

# Menghitung standard error dari koefisien
std_error = np.std(coefficients_bootstrap, axis=0)

# Membulatkan nilai standard error
std_error_rounded = np.round(std_error, 4)

print(std_error_rounded)


[[0.1638 0.3764 0.2469 0.7265 0.1698 0.2548]]


In [34]:
# Menghitung nilai Wald
wald_values = (coefficients / std_error)
# Menampilkan hasil uji Wald
print("Nilai Koefisien:")
for i, feature in enumerate(selected_features):
    coef_rounded = round(coefficients[0][i], 4)
    print(f"{feature}: {coef_rounded}")
print("\nNilai Wald:")
print(wald_values.round(4))

Nilai Koefisien:
R1006B: -0.7225
R1005C: -1.3511
R604A: -0.8929
R809D: 2.162
R1401A: -0.4575
R1001A: -0.5996

Nilai Wald:
[[-4.4113 -3.5899 -3.6167  2.9761 -2.6944 -2.3537]]


In [35]:
print("Nilai Odds Ratio:")
for i, feature in enumerate(selected_features):
    coef_exp = math.exp(coefficients[0][i])  # Menghitung nilai eksponensial
    print(f"{feature}: {coef_exp}")

Nilai Odds Ratio:
R1006B: 0.48554446010525343
R1005C: 0.2589584024942433
R604A: 0.40944693435460267
R809D: 8.68871956237254
R1401A: 0.6328790974153835
R1001A: 0.5490122463559747


In [36]:
# Menggunakan model terbaik untuk memperoleh probabilitas prediksi
y_pred_proba = best_model.predict_proba(x_test[selected_features])

# Menentukan threshold
threshold = 0.5

# Mengklasifikasikan prediksi berdasarkan threshold
y_pred = [1 if prob > threshold else 0 for prob in y_pred_proba[:, 1]]

# Menampilkan hasil prediksi
print("Predictions on Testing Data:")
print(y_pred)

# Evaluasi akurasi pada data testing
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy on Testing Data: {accuracy * 100:.2f}%')

# Menghitung confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Menampilkan confusion matrix
print("Confusion Matrix:")
print(cm)

Predictions on Testing Data:
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Accuracy on Testing Data: 81.21%
Confusion Matrix:
[[  5  28]
 [  3 129]]


In [37]:
# Menghitung presisi
precision = precision_score(y_test, y_pred)

# Menghitung recall
recall = recall_score(y_test, y_pred)

# Menghitung F-Measure
f_measure = f1_score(y_test, y_pred)

# Menghitung confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Menghitung Specificity
specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])

# Menampilkan hasil
print("Precision:", precision)
print("Recall:", recall)
print("F-Measure:", f_measure)
print("Specificity:", specificity)

Precision: 0.821656050955414
Recall: 0.9772727272727273
F-Measure: 0.8927335640138409
Specificity: 0.15151515151515152
